# Preparing the training images

### Untarring the training images

In [10]:
import tarfile
import os
from tqdm import tqdm
import shutil

In [13]:
directory = "./images/ILSVRC2010_images_train/"
# filenames = [f for root, dirs, f in os.walk(directory)][0]
filenames = [f for f in os.listdir(directory) if f.endswith('.tar')]

In [9]:
for i in tqdm(filenames):
    with tarfile.open(f"{directory}/{i}", 'r') as tar:
        tar.extractall(path="./images/training_images/")

100%|██████████| 1000/1000 [06:20<00:00,  2.63it/s]


In [14]:
# deleting the folder with the tar files to free up space
# using shutil instead of  os, apparently safer than using the shell
if os.path.exists(directory):
    shutil.rmtree(directory)
    print(f"Deleted directory: {directory}")
else:
    print(f"Directory does not exist: {directory}")

Deleted directory: ./images/ILSVRC2010_images_train/


### Replacing patch images

In [21]:
patch_images_train = "./images/patch_images/train/"

filepath_all_patch_train = []
for i in os.listdir(patch_images_train):
    for j in os.listdir(patch_images_train + i):
        filepath_all_patch_train.append(patch_images_train + i + j)

#print(filepath_all_patch_train)